In [1]:
import pandas as pd
from pathlib import Path

data_dir = Path("./CSV-data")
csv_files = sorted([f for f in data_dir.glob("*.csv")])
dfs = []
for csv_file in csv_files:
    print(f"processing file: {csv_file}")
    try:
        df = pd.read_csv(csv_file)
        dfs.append(df)
    except Exception as e:
        print(f"Error parsing {csv_file}: {str(e)}")

processing file: CSV-data/chunk-data.csv
processing file: CSV-data/relation__cityGroup.csv


In [2]:
import openai
import os
api_key_path = "./apiKey.txt"

with open(api_key_path, "r") as f:
  api_key = ' '.join(f.readlines())
  openai.api_key = api_key

os.environ['OPENAI_API_KEY'] = openai.api_key

In [3]:
tableinfo_dir = "LEI_TableInfo"
!mkdir {tableinfo_dir}

In [44]:
from llama_index.core.program import LLMTextCompletionProgram
from llama_index.core.bridge.pydantic import BaseModel, Field
from llama_index.llms.openai import OpenAI


class TableInfo(BaseModel):
    """Information regarding a structured table."""

    table_name: str = Field(
        ..., description="table name (must be underscores and NO spaces)"
    )
    table_summary: str = Field(
        ..., description="short, concise summary/caption of the table"
    )


prompt_str = """\
Give me a summary of the table with the following JSON format.

- The table name must be unique to the table and describe it while being concise. 
- Do NOT output a generic table name (e.g. table, my_table).

Do NOT make the table name one of the following: {exclude_table_name_list}

Table:
{table_str}

Summary: """

program = LLMTextCompletionProgram.from_defaults(
    output_cls=TableInfo,
    llm=OpenAI(model="gpt-3.5-turbo"),
    prompt_template_str=prompt_str,
)

In [31]:
import json


def _get_tableinfo_with_index(idx: int) -> str:
    results_gen = Path(tableinfo_dir).glob(f"{idx}_*")
    results_list = list(results_gen)
    print(results_list)
    if len(results_list) == 0:
        return None
    elif len(results_list) == 1:
        path = results_list[0]
        return TableInfo.parse_file(path)
    else:
        raise ValueError(
            f"More than one file matching index: {list(results_gen)}"
        )

In [45]:
table_names = set()
table_infos = []
for idx, df in enumerate(dfs):
    table_info = _get_tableinfo_with_index(idx)
    print(table_info)
    if table_info:
        table_infos.append(table_info)
    else:
        while True:
            df_str = df.head(10).to_csv()
            table_info = program(
                table_str=df_str,
                exclude_table_name_list=str(list(table_names)),
            )
            table_name = table_info.table_name
            print(f"Processed table: {table_name}")
            if table_name not in table_names:
                table_names.add(table_name)
                break
            else:
                # try again
                print(f"Table name {table_name} already exists, trying again.")
                pass

        out_file = f"{tableinfo_dir}/{idx}_{table_name}.json"
        json.dump(table_info.dict(), open(out_file, "w"))
        table_infos.append(table_info)

[]
None
Processed table: Entity Information Table
[]
None
Processed table: Relationship Information Table


In [46]:
table_infos

[TableInfo(table_name='Entity Information Table', table_summary='This table contains detailed information about various entities including legal names, addresses, registration details, entity categories, legal forms, entity status, creation dates, and legal entity events.'),
 TableInfo(table_name='Relationship Information Table', table_summary='Summary of relationships between nodes, including start and end node IDs, relationship type, status, periods, qualifiers, quantifiers, registration details, and validation sources.')]

In [47]:
table_infos[1].table_summary = "This table contains information about relationships between nodes, including start and end node IDs, relationship types, status, periods, qualifiers, quantifiers, registration details, and validation sources.' 'Relationship.EndNode.NodeID' column is the foreign key" 

In [48]:
table_infos

[TableInfo(table_name='Entity Information Table', table_summary='This table contains detailed information about various entities including legal names, addresses, registration details, entity categories, legal forms, entity status, creation dates, and legal entity events.'),
 TableInfo(table_name='Relationship Information Table', table_summary="This table contains information about relationships between nodes, including start and end node IDs, relationship types, status, periods, qualifiers, quantifiers, registration details, and validation sources.' 'Relationship.EndNode.NodeID' column is the foreign key")]

In [8]:
from sqlalchemy import (
    create_engine,
    MetaData,
    Table,
    Column,
    String,
    Integer,
)
import re

In [9]:
# Function to create a sanitized column name
def sanitize_column_name(col_name):
    # Remove special characters and replace spaces with underscores
    return re.sub(r"\W+", "_", col_name)

In [10]:
# Function to create a table from a DataFrame using SQLAlchemy
def create_table_from_dataframe(
    df: pd.DataFrame, table_name: str, engine, metadata_obj
):
    # Sanitize column names
    sanitized_columns = {col: sanitize_column_name(col) for col in df.columns}
    df = df.rename(columns=sanitized_columns)

    # Dynamically create columns based on DataFrame columns and data types
    columns = [
        Column(col, String if dtype == "object" else Integer)
        for col, dtype in zip(df.columns, df.dtypes)
    ]

    # Create a table with the defined columns
    table = Table(table_name, metadata_obj, *columns)

    # Create the table in the database
    metadata_obj.create_all(engine)

    # Insert data from DataFrame into the table
    with engine.connect() as conn:
        for _, row in df.iterrows():
            insert_stmt = table.insert().values(**row.to_dict())
            conn.execute(insert_stmt)
        conn.commit()

In [11]:
engine = create_engine("sqlite:///:memory:")
metadata_obj = MetaData()
for idx, df in enumerate(dfs):
    tableinfo = _get_tableinfo_with_index(idx)
    print(f"Creating table: {tableinfo.table_name}")
    create_table_from_dataframe(df, tableinfo.table_name, engine, metadata_obj)

[PosixPath('LEI_TableInfo/0_Entity Information Table.json')]
Creating table: Entity Information Table
[PosixPath('LEI_TableInfo/1_Relationship Information Table.json')]
Creating table: Relationship Information Table


In [49]:
from llama_index.core.objects import (
    SQLTableNodeMapping,
    ObjectIndex,
    SQLTableSchema,
)
from llama_index.core import SQLDatabase, VectorStoreIndex

sql_database = SQLDatabase(engine)

table_node_mapping = SQLTableNodeMapping(sql_database)
table_schema_objs = [
    SQLTableSchema(table_name=t.table_name, context_str=t.table_summary)
    for t in table_infos
]  # add a SQLTableSchema for each table

obj_index = ObjectIndex.from_objects(
    table_schema_objs,
    table_node_mapping,
    VectorStoreIndex,
)
obj_retriever = obj_index.as_retriever(similarity_top_k=2)

In [14]:
from llama_index.core.retrievers import SQLRetriever
from typing import List
from llama_index.core.query_pipeline import FnComponent

sql_retriever = SQLRetriever(sql_database)


def get_table_context_str(table_schema_objs: List[SQLTableSchema]):
    """Get table context string."""
    context_strs = []
    for table_schema_obj in table_schema_objs:
        table_info = sql_database.get_single_table_info(
            table_schema_obj.table_name
        )
        if table_schema_obj.context_str:
            table_opt_context = " The table description is: "
            table_opt_context += table_schema_obj.context_str
            table_info += table_opt_context

        context_strs.append(table_info)
    return "\n\n".join(context_strs)


table_parser_component = FnComponent(fn=get_table_context_str)

In [51]:
from llama_index.core.prompts.default_prompts import DEFAULT_TEXT_TO_SQL_PROMPT
from llama_index.core import PromptTemplate
from llama_index.core.llms import ChatResponse


def parse_response_to_sql(response: ChatResponse) -> str:
    """Parse response to SQL."""
    response = response.message.content
    sql_query_start = response.find("SQLQuery:")
    if sql_query_start != -1:
        response = response[sql_query_start:]
        if response.startswith("SQLQuery:"):
            response = response[len("SQLQuery:") :]
    sql_result_start = response.find("SQLResult:")
    if sql_result_start != -1:
        response = response[:sql_result_start]
    return response.strip().strip("```").strip()


sql_parser_component = FnComponent(fn=parse_response_to_sql)

text2sql_prompt = DEFAULT_TEXT_TO_SQL_PROMPT.partial_format(
    dialect=engine.dialect.name     # sqlite
)
print(text2sql_prompt.template)

Given an input question, first create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer. You can order the results by a relevant column to return the most interesting examples in the database.

Never query for all the columns from a specific table, only ask for a few relevant columns given the question.

Pay attention to use only the column names that you can see in the schema description. Be careful to not query for columns that do not exist. Pay attention to which column is in which table. Also, qualify column names with the table name when needed. You are required to use the following format, each taking one line:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of the SQLQuery
Answer: Final answer here

Only use tables listed below.
{schema}

Question: {query_str}
SQLQuery: 


In [52]:
response_synthesis_prompt_str = (
    "Given an input question, synthesize a response from the query results.\n"
    "Query: {query_str}\n"
    "SQL: {sql_query}\n"
    "SQL Response: {context_str}\n"
    "Response: "
)
response_synthesis_prompt = PromptTemplate(
    response_synthesis_prompt_str,
)

In [53]:
llm=OpenAI(model="gpt-3.5-turbo")

In [54]:
from llama_index.core.query_pipeline import (
    QueryPipeline as QP,
    Link,
    InputComponent,
    CustomQueryComponent,
)

qp = QP(
    modules={
        "input": InputComponent(),
        "table_retriever": obj_retriever,
        "table_output_parser": table_parser_component,
        "text2sql_prompt": text2sql_prompt,
        "text2sql_llm": llm,
        "sql_output_parser": sql_parser_component,
        "sql_retriever": sql_retriever,
        "response_synthesis_prompt": response_synthesis_prompt,
        "response_synthesis_llm": llm,
    },
    verbose=True,
)

In [55]:
qp.add_chain(["input", "table_retriever", "table_output_parser"])
qp.add_link("input", "text2sql_prompt", dest_key="query_str")
qp.add_link("table_output_parser", "text2sql_prompt", dest_key="schema")
qp.add_chain(
    ["text2sql_prompt", "text2sql_llm", "sql_output_parser", "sql_retriever"]
)
qp.add_link(
    "sql_output_parser", "response_synthesis_prompt", dest_key="sql_query"
)
qp.add_link(
    "sql_retriever", "response_synthesis_prompt", dest_key="context_str"
)
qp.add_link("input", "response_synthesis_prompt", dest_key="query_str")
qp.add_link("response_synthesis_prompt", "response_synthesis_llm")

In [56]:
from llama_index.core import VectorStoreIndex, load_index_from_storage
from sqlalchemy import text
from llama_index.core.schema import TextNode
from llama_index.core import StorageContext
import os
from pathlib import Path
from typing import Dict


def index_all_tables(
    sql_database: SQLDatabase, table_index_dir: str = "table_index_dir"
) -> Dict[str, VectorStoreIndex]:
    """Index all tables."""
    if not Path(table_index_dir).exists():
        os.makedirs(table_index_dir)

    vector_index_dict = {}
    engine = sql_database.engine
    for table_name in sql_database.get_usable_table_names():
        print(f"Indexing rows in table: {table_name}")
        if not os.path.exists(f"{table_index_dir}/{table_name}"):
            # get all rows from table
            with engine.connect() as conn:
                cursor = conn.execute(text(f'SELECT * FROM "{table_name}"'))
                result = cursor.fetchall()
                row_tups = []
                for row in result:
                    row_tups.append(tuple(row))

            # index each row, put into vector store index
            nodes = [TextNode(text=str(t)) for t in row_tups]

            # put into vector store index (use OpenAIEmbeddings by default)
            index = VectorStoreIndex(nodes)

            # save index
            index.set_index_id("vector_index")
            index.storage_context.persist(f"{table_index_dir}/{table_name}")
        else:
            # rebuild storage context
            storage_context = StorageContext.from_defaults(
                persist_dir=f"{table_index_dir}/{table_name}"
            )
            # load index
            index = load_index_from_storage(
                storage_context, index_id="vector_index"
            )
        vector_index_dict[table_name] = index

    return vector_index_dict


vector_index_dict = index_all_tables(sql_database)

Indexing rows in table: Entity Information Table
Indexing rows in table: Relationship Information Table


In [57]:
test_retriever = vector_index_dict["Entity Information Table"].as_retriever(
    similarity_top_k=1
)

In [58]:
nodes = test_retriever.retrieve("Citygroup")
print(nodes[0].get_content())

(1518183, '6R01V5A3ZABX0SW8LX55', 'Goldman Sachs Developing Markets Real Estate Coöperatief U.A.', 'nl', None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, 'nl', 'Herikerbergweg 88', None, None, None, None, None, 'Amsterdam', 'NL-NH', 'NL', '1101 CM', 'nl', 'Herikerbergweg 88', None, None, None, None, None, 'Amsterdam', 'NL-NH', 'NL', '1101 CM', None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, 'RA000463', '34279914', 'NL', 'GENERAL', None, 'NFFH', None, 'ACTIVE', '2007-08-06T00:00:00+00:00', None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, '2012-10-30T19:02:00+

In [60]:
from llama_index.core.retrievers import SQLRetriever
from typing import List
from llama_index.core.query_pipeline import FnComponent

sql_retriever = SQLRetriever(sql_database)


def get_table_context_and_rows_str(
    query_str: str, table_schema_objs: List[SQLTableSchema]
):
    """Get table context string."""
    context_strs = []
    for table_schema_obj in table_schema_objs:
        # first append table info + additional context
        table_info = sql_database.get_single_table_info(
            table_schema_obj.table_name
        )
        if table_schema_obj.context_str:
            table_opt_context = " The table description is: "
            table_opt_context += table_schema_obj.context_str
            table_info += table_opt_context

        # also lookup vector index to return relevant table rows
        vector_retriever = vector_index_dict[
            table_schema_obj.table_name
        ].as_retriever(similarity_top_k=2)
        relevant_nodes = vector_retriever.retrieve(query_str)
        if len(relevant_nodes) > 0:
            table_row_context = "\nHere are some relevant example rows (values in the same order as columns above)\n"
            for node in relevant_nodes:
                table_row_context += str(node.get_content()) + "\n"
            table_info += table_row_context

        context_strs.append(table_info)
    return "\n\n".join(context_strs)


table_parser_component = FnComponent(fn=get_table_context_and_rows_str)

In [61]:
from llama_index.core.query_pipeline import (
    QueryPipeline as QP,
    Link,
    InputComponent,
    CustomQueryComponent,
)

qp = QP(
    modules={
        "input": InputComponent(),
        "table_retriever": obj_retriever,
        "table_output_parser": table_parser_component,
        "text2sql_prompt": text2sql_prompt,
        "text2sql_llm": llm,
        "sql_output_parser": sql_parser_component,
        "sql_retriever": sql_retriever,
        "response_synthesis_prompt": response_synthesis_prompt,
        "response_synthesis_llm": llm,
    },
    verbose=True,
)

In [62]:
qp.add_link("input", "table_retriever")
qp.add_link("input", "table_output_parser", dest_key="query_str")
qp.add_link(
    "table_retriever", "table_output_parser", dest_key="table_schema_objs"
)
qp.add_link("input", "text2sql_prompt", dest_key="query_str")
qp.add_link("table_output_parser", "text2sql_prompt", dest_key="schema")
qp.add_chain(
    ["text2sql_prompt", "text2sql_llm", "sql_output_parser", "sql_retriever"]
)
qp.add_link(
    "sql_output_parser", "response_synthesis_prompt", dest_key="sql_query"
)
qp.add_link(
    "sql_retriever", "response_synthesis_prompt", dest_key="context_str"
)
qp.add_link("input", "response_synthesis_prompt", dest_key="query_str")
qp.add_link("response_synthesis_prompt", "response_synthesis_llm")

In [27]:
response = qp.run(
    query="What is the LEI for CITYGROUP?"
)
print(str(response))

> Running module input with input: 
query: What is the LEI for CITYGROUP?

> Running module table_retriever with input: 
input: What is the LEI for CITYGROUP?

> Running module table_output_parser with input: 
query_str: What is the LEI for CITYGROUP?
table_schema_objs: [SQLTableSchema(table_name='Entity Information Table', context_str='This table contains information about various entities including legal names, addresses, registration details, and conformity status...

> Running module text2sql_prompt with input: 
query_str: What is the LEI for CITYGROUP?
schema: Table 'Entity Information Table' has columns: Unnamed_0 (INTEGER), LEI (VARCHAR), Entity_LegalName (VARCHAR), Entity_LegalName_xmllang (VARCHAR), Entity_OtherEntityNames_OtherEntityName_1 (VARCHAR), E...

> Running module text2sql_llm with input: 
messages: Given an input question, first create a syntactically correct sqlite query to run, then look at the results of the query and return the answer. You can order the results 

In [63]:
response = qp.run(
    query="How many relation are directly consolidated by citygroup inc?"
)
print(str(response))

> Running module input with input: 
query: How many relation are directly consolidated by citygroup inc?

> Running module table_retriever with input: 
input: How many relation are directly consolidated by citygroup inc?

> Running module table_output_parser with input: 
query_str: How many relation are directly consolidated by citygroup inc?
table_schema_objs: [SQLTableSchema(table_name='Relationship Information Table', context_str="This table contains information about relationships between nodes, including start and end node IDs, relationship types, statu...

> Running module text2sql_prompt with input: 
query_str: How many relation are directly consolidated by citygroup inc?
schema: Table 'Relationship Information Table' has columns: Unnamed_0 (INTEGER), Relationship_StartNode_NodeID (VARCHAR), Relationship_StartNode_NodeIDType (VARCHAR), Relationship_EndNode_NodeID (VARCHAR), Re...

> Running module text2sql_llm with input: 
messages: Given an input question, first create a syntac

In [64]:
response = qp.run(
    query="what is the foreign key for LEI in relationship table"
)
print(str(response))

> Running module input with input: 
query: what is the foreign key for LEI in relationship table

> Running module table_retriever with input: 
input: what is the foreign key for LEI in relationship table

> Running module table_output_parser with input: 
query_str: what is the foreign key for LEI in relationship table
table_schema_objs: [SQLTableSchema(table_name='Relationship Information Table', context_str="This table contains information about relationships between nodes, including start and end node IDs, relationship types, statu...

> Running module text2sql_prompt with input: 
query_str: what is the foreign key for LEI in relationship table
schema: Table 'Relationship Information Table' has columns: Unnamed_0 (INTEGER), Relationship_StartNode_NodeID (VARCHAR), Relationship_StartNode_NodeIDType (VARCHAR), Relationship_EndNode_NodeID (VARCHAR), Re...

> Running module text2sql_llm with input: 
messages: Given an input question, first create a syntactically correct sqlite query to 

In [65]:
response = qp.run(
    query="what are the unique result for RelationshipType in Relationship Information Table"
)
print(str(response))

> Running module input with input: 
query: what are the unique result for RelationshipType in Relationship Information Table

> Running module table_retriever with input: 
input: what are the unique result for RelationshipType in Relationship Information Table

> Running module table_output_parser with input: 
query_str: what are the unique result for RelationshipType in Relationship Information Table
table_schema_objs: [SQLTableSchema(table_name='Relationship Information Table', context_str="This table contains information about relationships between nodes, including start and end node IDs, relationship types, statu...

> Running module text2sql_prompt with input: 
query_str: what are the unique result for RelationshipType in Relationship Information Table
schema: Table 'Relationship Information Table' has columns: Unnamed_0 (INTEGER), Relationship_StartNode_NodeID (VARCHAR), Relationship_StartNode_NodeIDType (VARCHAR), Relationship_EndNode_NodeID (VARCHAR), Re...

> Running module te

In [66]:
response = qp.run(
    query="Based on the LEI how many RelationshipType in Relationship Information Table are IS_DIRECTLY_CONSOLIDATED_BY with CityGroup Inc"
)
print(str(response))

> Running module input with input: 
query: Based on the LEI how many RelationshipType in Relationship Information Table are IS_DIRECTLY_CONSOLIDATED_BY with CityGroup Inc

> Running module table_retriever with input: 
input: Based on the LEI how many RelationshipType in Relationship Information Table are IS_DIRECTLY_CONSOLIDATED_BY with CityGroup Inc

> Running module table_output_parser with input: 
query_str: Based on the LEI how many RelationshipType in Relationship Information Table are IS_DIRECTLY_CONSOLIDATED_BY with CityGroup Inc
table_schema_objs: [SQLTableSchema(table_name='Relationship Information Table', context_str="This table contains information about relationships between nodes, including start and end node IDs, relationship types, statu...

> Running module text2sql_prompt with input: 
query_str: Based on the LEI how many RelationshipType in Relationship Information Table are IS_DIRECTLY_CONSOLIDATED_BY with CityGroup Inc
schema: Table 'Relationship Information Table' h

In [67]:
response = qp.run(
    query="how many RelationshipType in Relationship Information Table are IS_DIRECTLY_CONSOLIDATED_BY"
)
print(str(response))

> Running module input with input: 
query: how many RelationshipType in Relationship Information Table are IS_DIRECTLY_CONSOLIDATED_BY

> Running module table_retriever with input: 
input: how many RelationshipType in Relationship Information Table are IS_DIRECTLY_CONSOLIDATED_BY

> Running module table_output_parser with input: 
query_str: how many RelationshipType in Relationship Information Table are IS_DIRECTLY_CONSOLIDATED_BY
table_schema_objs: [SQLTableSchema(table_name='Relationship Information Table', context_str="This table contains information about relationships between nodes, including start and end node IDs, relationship types, statu...

> Running module text2sql_prompt with input: 
query_str: how many RelationshipType in Relationship Information Table are IS_DIRECTLY_CONSOLIDATED_BY
schema: Table 'Relationship Information Table' has columns: Unnamed_0 (INTEGER), Relationship_StartNode_NodeID (VARCHAR), Relationship_StartNode_NodeIDType (VARCHAR), Relationship_EndNode_Node

In [68]:
response = qp.run(
    query="Based on the LEI '6SHGI4ZSSLCXXQSBB395' how many RelationshipType in Relationship Information Table are IS_DIRECTLY_CONSOLIDATED_BY with CityGroup Inc"
)
print(str(response))

> Running module input with input: 
query: Based on the LEI '6SHGI4ZSSLCXXQSBB395' how many RelationshipType in Relationship Information Table are IS_DIRECTLY_CONSOLIDATED_BY with CityGroup Inc

> Running module table_retriever with input: 
input: Based on the LEI '6SHGI4ZSSLCXXQSBB395' how many RelationshipType in Relationship Information Table are IS_DIRECTLY_CONSOLIDATED_BY with CityGroup Inc

> Running module table_output_parser with input: 
query_str: Based on the LEI '6SHGI4ZSSLCXXQSBB395' how many RelationshipType in Relationship Information Table are IS_DIRECTLY_CONSOLIDATED_BY with CityGroup Inc
table_schema_objs: [SQLTableSchema(table_name='Relationship Information Table', context_str="This table contains information about relationships between nodes, including start and end node IDs, relationship types, statu...

> Running module text2sql_prompt with input: 
query_str: Based on the LEI '6SHGI4ZSSLCXXQSBB395' how many RelationshipType in Relationship Information Table are IS_

In [69]:
response = qp.run(
    query="Based on the LEI '6SHGI4ZSSLCXXQSBB395' how many RelationshipType in Relationship Information Table are IS_DIRECTLY_CONSOLIDATED_BY"
)
print(str(response))

> Running module input with input: 
query: Based on the LEI '6SHGI4ZSSLCXXQSBB395' how many RelationshipType in Relationship Information Table are IS_DIRECTLY_CONSOLIDATED_BY

> Running module table_retriever with input: 
input: Based on the LEI '6SHGI4ZSSLCXXQSBB395' how many RelationshipType in Relationship Information Table are IS_DIRECTLY_CONSOLIDATED_BY

> Running module table_output_parser with input: 
query_str: Based on the LEI '6SHGI4ZSSLCXXQSBB395' how many RelationshipType in Relationship Information Table are IS_DIRECTLY_CONSOLIDATED_BY
table_schema_objs: [SQLTableSchema(table_name='Relationship Information Table', context_str="This table contains information about relationships between nodes, including start and end node IDs, relationship types, statu...

> Running module text2sql_prompt with input: 
query_str: Based on the LEI '6SHGI4ZSSLCXXQSBB395' how many RelationshipType in Relationship Information Table are IS_DIRECTLY_CONSOLIDATED_BY
schema: Table 'Relationship Info